# RDP #9 Initiators and Deleters
[documentation](https://stonesoup.readthedocs.io/en/v0.1b6/auto_tutorials/09_Initiators_%26_Deleters.html)

In [ ]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
from importlib import reload  # Python 3.4+
from typing import Tuple
import itertools
from matplotlib import pyplot as plt
from math import ceil
from ordered_set import OrderedSet

import dateutil
from pymap3d import geodetic2enu

import sys
sys.path.append('C:/Users/ttrinter/git_repo/cspeed/data_common')
sys.path.append('../../..')
import data_functions as dfunc
import visualizations as v
from ttt_ss_funcs import generate_timestamps, ADSBTruthReader, CSVReaderXY, CSVReaderPolar, plot_all, CSVClutterReaderXY, group_plots
from ttt_ss_funcs import RDPandClutterInitiator

from stonesoup.reader import DetectionReader, GroundTruthReader
from stonesoup.base import Property
from stonesoup.types.detection import Detection, Clutter
from stonesoup.plotter import AnimatedPlotterly, Plotter

from stonesoup.base import Property
from stonesoup.buffered_generator import BufferedGenerator
from stonesoup.functions import cart2sphere, sphere2cart
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange
from stonesoup.types.angle import Bearing
from stonesoup.types.detection import Detection
from stonesoup.types.groundtruth import GroundTruthState, GroundTruthPath

# Tracker Imports
from stonesoup.types.state import GaussianState

plot_type = 'static' # or 'animated'

sensor_positions = { 'RDU103': (51.52126391, 5.85862734)}

METERS_in_NM = 1852


In [ ]:
target_address1 = 10537421 # plane #1
target_address2 = 10610889 # plane #

## Pass Number
All of the examples are looking at data once/second. That matches up the frequency of new plots. However, for our purposes, it may be better to look per radar pass, assuming that the radar will not see a target more than once per pass.

I should be able to set the pass number on the plots and clutter using the cat 34 sector messages.

## Save/Read  to/from CSV

In [ ]:
from math import pi
data_dir = 'C:/Users/ttrinter/git_repo/Stone-Soup/data'

# Now there are 2 ADSB Files
adsb_data = pd.DataFrame()
adsb_files = [f'{data_dir}/adsb_straight.csv', f'{data_dir}/adsb_straight2.csv']
for file in adsb_files:
    this_data = pd.read_csv(file)
    this_data['timestamp'] = pd.to_datetime(this_data['timestamp'], errors='coerce')
    this_data = this_data.loc[~this_data['timestamp'].isna()]
    this_data['timestamp'] = pd.to_datetime(this_data['timestamp'], errors='coerce')
    this_data['timestamp'] = this_data['timestamp'].dt.tz_localize(None)

    adsb_data = pd.concat([adsb_data, this_data])

# ADSB- first 3 min
# adsb_file = f'{data_dir}/adsb3.csv'
# adsb_data = pd.read_csv(adsb_file)

rdp_file = f'{data_dir}/rdp_straight.csv'
# rdp_straight['timestamp'] = pd.to_datetime(rdp_straight['timestamp'], errors='coerce')
# rdp_straight['theta_rad'] = np.deg2rad(rdp_straight.theta)
# rdp_straight.loc[rdp_straight.theta_rad>2*pi, 'theta_rad'] = rdp_straight.loc[rdp_straight.theta_rad>2*pi, 'theta_rad'] - 2*pi 

# rdp_straight = rdp_straight.loc[~rdp_straight['timestamp'].isna()]
# rdp_straight.to_csv(rdp_file, index=False)
# rdp_data = pd.read_csv(rdp_file)
# rdp_data['timestamp'] = pd.to_datetime(rdp_data['timestamp'], errors='coerce')
# rdp_data['timestamp'] = rdp_data['timestamp'].dt.tz_localize(None)
# rdp_data['timestamp'] = rdp_data['timestamp'].astype('datetime64[us]')
# rdp_data['timestamp'] = rdp_data['timestamp'].round('ms')


# Matched Plots
matched_csv1 = f'{data_dir}/rdp_matched.csv'
matched_csv2 = f'{data_dir}/rdp_matched2.csv'
matched_csv3min = f'{data_dir}/rdp_matched3.csv'
matched_rdp_file = matched_csv2

rdp_matched = pd.read_csv(matched_rdp_file)
rdp_matched['timestamp'] = pd.to_datetime(rdp_matched['timestamp'], errors='coerce')
rdp_matched['timestamp'] = rdp_matched['timestamp'].dt.tz_localize(None)
rdp_matched['timestamp'] = rdp_matched['timestamp'].astype('datetime64[us]')
rdp_matched['timestamp'] = rdp_matched['timestamp'].round('ms')

start_time = rdp_matched['timestamp'].min()
end_time = rdp_matched['timestamp'].max()

clutter_file = f'{data_dir}/sample_clutter.csv'
# clutter_file = f'{data_dir}/clutter3.csv'
clutter_data = pd.read_csv(clutter_file)
clutter_data['timestamp'] = pd.to_datetime(clutter_data['timestamp'], errors='coerce')
clutter_data = clutter_data.loc[~clutter_data['timestamp'].isna()]
clutter_data['timestamp'] = pd.to_datetime(clutter_data['timestamp'], errors='coerce')
clutter_data['timestamp'] = clutter_data['timestamp'].dt.tz_localize(None)

print(f'ADSB: {len(adsb_data)}')
# print(f'RDP: {len(rdp_data)}')

In [ ]:
print(f'{clutter_data.timestamp.min()} : {clutter_data.timestamp.max()}')

In [ ]:
v.scatter_targets(clutter_data)
plt.suptitle('Clutter')

## Matched Data Set
To make things even simpler, I'll grab the set of matched data for this test plane. Then most of the plots should be "true" detections. Let's see how the tracker does with that.

In [ ]:
target_address1 = 10537421 # plane #1
target_address2 = 10610889 # plane #
target_addresses = [target_address1, target_address2]
file_dir = 'C:/Users/ttrinter/OneDrive - cspeed.com (1)/Documents/Data/Travis/2024-07-17'
matched_file = '20240717_Travis_matched_rdp_61.xlsx'
all_matched_data = pd.read_excel(f'{file_dir}/{matched_file}')
matched_data = all_matched_data.loc[(all_matched_data.target_address.isin(target_addresses)) &
                                (all_matched_data.close_enough==True) & 
                                (all_matched_data.timestamp_adsb>=start_time) & 
                                (all_matched_data.timestamp_adsb<=end_time)]
matched_data.head()

In [ ]:
matched_plot = v.plot_target_match2(matching=matched_data, 
                                    target_address=target_address1, 
                                    plot_show=True, 
                                    pd_loc='title')  

In [ ]:
matched_plot = v.plot_target_match2(matching=matched_data, 
                                    target_address=target_address2, 
                                    plot_show=True, 
                                    pd_loc='title')  

In [ ]:
# Plotting
fig, ax = plt.subplots()

# Group by the 'category' column and plot each group separately
for tgt, group in rdp_matched.groupby('target_address'):
    ax.scatter(group['x'], group['y'], label=tgt, marker="+")

# Add labels and title
plt.legend()
plt.grid()
plt.title("Matched RDP Plots")

In [ ]:
adsb = ADSBTruthReader.multiple_ground_truth_reader(adsb_files)
# adsb = ADSBTruthReader.single_ground_truth_reader(adsb_file)

In [ ]:
plotter = Plotter()
plotter.plot_ground_truths(adsb, 
                        mapping=[0, 2], 
                        markersize = 5,
                        marker = 's', 
                        markerfacecolor = 'none', 
                        alpha = 0.2)
plt.grid()
plt.title("Two Targets of Opportunity")

In [ ]:
# Detections
matched_xy = CSVReaderXY(matched_rdp_file)
matched_polar = CSVReaderPolar(matched_rdp_file)

# Cutter
clutter = CSVClutterReaderXY(clutter_file)

dets = [next(iter(detection[1])) for detection in matched_xy.detections_gen()]
cluts = [next(iter(detection[1])) for detection in clutter.detections_gen()]

# Combine detections with clutter
all_measurements = dets + cluts
all_measurements.sort(key=lambda obj: obj.timestamp)

plot_all(start_time, 
         end_time,
         all_measurements=all_measurements, 
         adsb=adsb, 
         plot_type='static')

## From Tutorial #9

In [ ]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
                                               ConstantVelocity

from stonesoup.measures import Mahalanobis
from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.updater.kalman import KalmanUpdater

from stonesoup.predictor.kalman import ExtendedKalmanPredictor
from stonesoup.updater.kalman import ExtendedKalmanUpdater

from stonesoup.predictor.kalman import UnscentedKalmanPredictor
from stonesoup.updater.kalman import UnscentedKalmanUpdater

from stonesoup.types.track import Track
from stonesoup.hypothesiser.distance import DistanceHypothesiser

from stonesoup.dataassociator.neighbour import NearestNeighbour
from stonesoup.dataassociator.neighbour import GNNWith2DAssignment
from stonesoup.deleter.error import CovarianceBasedDeleter

from stonesoup.types.state import GaussianState
from stonesoup.initiator.simple import MultiMeasurementInitiator
from stonesoup.deleter.time import UpdateTimeDeleter


In [ ]:
# Measurement Model
np.random.seed(42)
default_variance = 50

measurement_model = LinearGaussian(
    ndim_state=4,   # Number of state dimensions (position and velocity in 2D)
    mapping=(0, 2), # Mapping measurement vector index to state index
    noise_covar=np.array([[default_variance, 0 ],  
                          [0, default_variance]])
    )  #Covariance matrix for Gaussian PDF

# Transition Model
q_const = 60
q_x = q_const
q_y = q_const
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])

## Creating a Tracker

In [ ]:
predictor = ExtendedKalmanPredictor(transition_model)
updater = ExtendedKalmanUpdater(measurement_model)

#Hypothesisers
hypothesiser = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=3)
# init_hypothesiser = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=2)

#Data Associators
data_associator = NearestNeighbour(hypothesiser)
# data_associator = GNNWith2DAssignment(hypothesiser)
# init_data_associator = GNNWith2DAssignment(init_hypothesiser)

# Deleter
# deleter =  CovarianceBasedDeleter(covar_trace_thresh=4)
deleter = UpdateTimeDeleter(time_since_update=timedelta(seconds=30), delete_last_pred=True)
# init_deleter = UpdateTimeDeleter(time_since_update=timedelta(seconds=15), delete_last_pred=True)

prior_state = GaussianState([0, q_x, 0, q_y], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)
# prior_state = GaussianState([0, 1, 0, 1], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)
initiator = MultiMeasurementInitiator(
    prior_state=prior_state, 
    measurement_model=measurement_model, 
    deleter=deleter, 
    data_associator=data_associator, 
    updater=updater, 
    min_points=2)

## Running the Tracker

In [ ]:
tracks = set()

grouped_sec, grouped_pass = group_plots(all_measurements)

# for n, detections in enumerate(grouped_sec):
for n, detections in enumerate(grouped_pass):    
    this_time = min(detections, key=lambda detct: detct.timestamp).timestamp
    this_time = this_time.replace(microsecond=0)

    # Calculate all hypothesis pairs and associate the elements in the best subset to the tracks.
    if len(detections)>0:
        hypotheses = data_associator.associate(tracks, detections, this_time)
        
        associated_detections = set()
        for track in tracks:
            hypothesis = hypotheses[track]
            if hypothesis.measurement:
                post = updater.update(hypothesis)
                track.append(post)
                associated_detections.add(hypothesis.measurement)
            else:
                track.append(hypothesis.prediction)

        tracks -= deleter.delete_tracks(tracks = tracks)
        tracks |= initiator.initiate(detections - associated_detections, this_time)
        
        print(f'{n}: {len(tracks)} tracks')

    
print(f'{len(tracks)} tracks')

## Plot the ground truth and measurements with clutter.

In [ ]:
plot_all(start_time,
         end_time,
         all_measurements=all_measurements, 
         adsb=adsb,
         tracks=tracks,
        #  plot_type='static')
        plot_type='animated')

### Create Track Initators and Delters

## Multi-Target-Tracker

In [ ]:
tracker = MultiTargetTracker(
    detector=detector,
    initiator=initiator,
    deleter=deleter,
    data_associator=data_associator,
    updater=updater,
)

In [ ]:
tracks = set()
detections = set()
# try:
for time, ctracks in tracker:
    # print(f'{time}:{len(tracks)}')
    tracks |= ctracks
    detections |= tracker.detector.detections
# except:
#     pass
len(tracks)

This tracker did a good job of following each of the tracks separately! However, the more vertical track was wierdly combined with what appears to be an entirely different track in another location and direction.

Similar to the previous analysis - after running the polar tracks, re-running these cartesian tracks no longer works! Clearly something is lingering after a run. Not all inputs are getting reset with a re-run. Need to sort that out!

It is also suspect that 14 correlated plots get passed and are not added to the first track. It is not clear what/why these plots are skipped. Need to investigate that too. Perhaps with a smaller dataset that only inlcudes the first 20 correlated plots or something.

Maybe the subsequent tutorials will address track deletions and initiation.


## Polar Coordinates
Trying again, but changing the process to read rho and theta and, maybe later also radial velocity.

In [ ]:
# Detections
meas_polar = CSVReaderPolar(matched_csv2)

dets = [next(iter(detection[1])) for detection in meas_polar.detections_gen()]
cluts = [next(iter(detection[1])) for detection in clutter.detections_gen()]

# Combine detections with clutter
all_measurements = dets + cluts
all_measurements.sort(key=lambda obj: obj.timestamp)


### Kalman Filtering Again...
Should be the same from here forward.

In [ ]:
# Transition Model
q_const = 35
q_x = q_const
q_y = q_const
default_variance=50

transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])

# measurement_model = CartesianToBearingRange(ndim_state=4, 
#                                             mapping=(0,2), 
#                                             noise_covar=np.array([[default_variance, 0 ],
#                                                                   [0, default_variance]]))

# Basic Kalman
predictor = KalmanPredictor(transition_model)
updater = KalmanUpdater(measurement_model)

#Extended Kalman
predictor = ExtendedKalmanUpdater(transition_model)
updater = ExtendedKalmanUpdater(measurement_model)

#Unscented Kalman
# predictor = UnscentedKalmanPredictor(transition_model)
# updater = UnscentedKalmanUpdater(measurement_model)  # Keep alpha as default = 0.5

hypothesiser = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=3)
data_associator = NearestNeighbour(hypothesiser)

# create a prior using the approximate start of the track
# prior = GaussianState([[track_start_x], [1], [track_start_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

prior1 = GaussianState([[track_start1_x], [1], [track_start1_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)
prior2 = GaussianState([[track_start2_x], [1], [track_start2_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

# create a prior using the location of the radar
# prior = GaussianState([[0], [q_const], [0], [q_const]], np.diag([default_variance, 0.5, default_variance, 0.5]), timestamp=start_time)

# Loop through the predict, hypothesise, associate and update steps.
# del(tracks)
if "tracks" in globals():
    del tracks

if "track" in globals():
    del(track)

tracks = {Track([prior1]), Track([prior2])}

grouped_sec, grouped_pass = group_plots(all_measurements)

# for n, measurements in enumerate(grouped_sec):
for n, measurements in enumerate(grouped_pass):    
    this_time = min(measurements, key=lambda meas: meas.timestamp).timestamp
    this_time = this_time.replace(microsecond=0)

    # Calculate all hypothesis pairs and associate the elements in the best subset to the tracks.
    if len(measurements)>0:
    # for n, measurements in enumerate(dets):
        try: 
            hypotheses = data_associator.associate(tracks,
                                                measurements,
                                                this_time)
            for track in tracks:
                hypothesis = hypotheses[track]
                if hypothesis.measurement:
                    post = updater.update(hypothesis)
                    track.append(post)
                else:  # When data associator says no detections are good enough, we'll keep the prediction
                    track.append(hypothesis.prediction)

        except:
            # print(f'{this_time}: {len(measurements)}: ERROR')
            continue


In [ ]:
plot_all(start_time,
         end_time,
         all_measurements=all_measurements, 
         adsb=adsb,
         tracks=tracks,
         plot_type='static')

It is unclear why the cartesian tracker worked reasonably well (besides continuing the track with a large change in direction and location) and the polar tracker didn't ever continue the track.

Leaving it for the next tutorial.

In [ ]:
len(grouped_sec)